# iNaturalist 2019 (FGVC6) – Plan

Objectives:
- Establish a fast, correct pipeline using official splits (train2019/val2019) and test2019.
- Start with a small baseline to validate data loading and submission format, then scale up.
- Use strong pretrained vision backbones (timm), AMP, cosine LR, label smoothing, balanced sampling, TTA, EMA.
- Train on train -> validate on val; final run on train+val -> predict test.

Milestones:
1) Env + GPU check; data inspection from JSONs; extract archives.
2) Minimal baseline: tf_efficientnet_b0@224 with light aug, single seed; sanity-check val acc and submission format.
3) Scale model/resolution: convnext_base@384 or tf_efficientnet_b4@380; stronger aug (RandAug/TrivialAug), cosine, warmup, EMA; class-balanced sampler.
4) TTA (e.g., flips, resize-crop variants). Save logits.
5) Train on train+val and generate test predictions; save submission.csv.
6) Error analysis on val: per-class, confidence bins; tweak aug/sampler if needed.

Validation & Tracking:
- Use official val2019.json as validation for baseline/tuning.
- Log per-epoch metrics and elapsed time; checkpoint best.
- Cache predictions/logits for quick re-use.

Ask Experts after plan and after baseline run for strategy validation and scaling advice.

In [2]:
import os, sys, json, tarfile, time, shutil, subprocess, math, random
from pathlib import Path
from collections import Counter, defaultdict

def run(cmd):
    print("$", " ".join(cmd), flush=True)
    try:
        out = subprocess.run(cmd, capture_output=True, text=True, check=False)
        print(out.stdout, flush=True)
        if out.stderr:
            print(out.stderr, file=sys.stderr, flush=True)
        return out.returncode
    except Exception as e:
        print("Command failed:", e, flush=True)
        return 1

print("Env check:")
run(['bash','-lc','nvidia-smi || true'])
print(f"Python: {sys.version}")
print("CWD:", os.getcwd())
print("Dir listing:")
for p in sorted(Path('.').glob('*')):
    try:
        sz = p.stat().st_size
    except Exception:
        sz = -1
    print(f" - {p.name} ({sz/1e6:.2f} MB)")

def load_json(fp):
    with open(fp,'r') as f:
        return json.load(f)

train_json = load_json('train2019.json')
val_json = load_json('val2019.json')
test_json = load_json('test2019.json')

# Categories mapping
categories = train_json.get('categories', [])
assert categories, 'No categories found in train2019.json'
cat_ids = [c['id'] for c in categories]
cat_ids_sorted = sorted(cat_ids)
catid2idx = {cid:i for i,cid in enumerate(cat_ids_sorted)}
idx2catid = {i:cid for cid,i in catid2idx.items()}
print(f"Classes: {len(categories)}; example cat ids: {cat_ids_sorted[:5]} -> idx {list(range(5))}")
Path('artifacts').mkdir(exist_ok=True)
Path('artifacts/catid2idx.json').write_text(json.dumps(catid2idx))
Path('artifacts/idx2catid.json').write_text(json.dumps(idx2catid))

# Image-id to filename maps
def build_img_map(j):
    imgs = j.get('images', [])
    return {img['id']: img['file_name'] for img in imgs}
train_img_map = build_img_map(train_json)
val_img_map = build_img_map(val_json)
test_img_map = build_img_map(test_json)
print(f"Train images: {len(train_img_map)}, Val images: {len(val_img_map)}, Test images: {len(test_img_map)}")

# Peek example file_name patterns
train_fns = list(train_img_map.values())[:3]
val_fns = list(val_img_map.values())[:3]
test_fns = list(test_img_map.values())[:3]
print("Sample train file_names:", train_fns)
print("Sample val file_names:", val_fns)
print("Sample test file_names:", test_fns)

# Class counts (train/val)
def class_counts(j):
    cnt = Counter()
    for ann in j.get('annotations', []):
        cnt[ann['category_id']] += 1
    return cnt
train_cls_cnt = class_counts(train_json)
val_cls_cnt = class_counts(val_json)
print(f"Train annotations: {sum(train_cls_cnt.values())}; unique classes in train: {len(train_cls_cnt)}")
print(f"Val annotations: {sum(val_cls_cnt.values())}; unique classes in val: {len(val_cls_cnt)}")
print("Top-5 frequent train classes:", train_cls_cnt.most_common(5))

# Verify mapping consistency on a few samples
sample_items = list(train_cls_cnt.items())[:5]
for cid,_ in sample_items:
    print(f"cat_id {cid} -> idx {catid2idx[cid]}")

# Extract archives using system tar (auto-detects compression).
# Marker placed next to archive to avoid dir assumptions.
def extract_with_tar(archive_path: str):
    marker = Path(archive_path + '.extracted_ok')
    if marker.exists():
        print(f"Already extracted (marker present): {archive_path}")
        return
    print(f"Extracting via tar: {archive_path}")
    t0 = time.time()
    # List a few entries first to validate archive
    rc = run(['bash','-lc', f"tar -tf {archive_path} | head -n 5"])
    if rc != 0:
        raise RuntimeError(f"Cannot list archive {archive_path}")
    # Extract
    rc = run(['bash','-lc', f"tar -xf {archive_path}"])
    if rc != 0:
        raise RuntimeError(f"Extraction failed for {archive_path}")
    marker.write_text('ok')
    print(f"Done in {(time.time()-t0)/60:.1f} min", flush=True)

extract_with_tar('train_val2019.tar.gz')
extract_with_tar('test2019.tar.gz')

# Sanity check a few paths exist (file_name is relative to CWD after extraction) 
def check_files(img_map, root='.'):
    root = Path(root)
    miss = 0
    keys = list(img_map.keys())
    if not keys:
        return 0
    for k in random.sample(keys, min(5, len(keys))):
        fn = img_map[k]
        fp = root / fn
        ok = fp.exists()
        print(f"Check: {fp} -> {ok}")
        if not ok: miss += 1
    return miss

print("Checking a few extracted files exist:")
miss_tv = check_files({**train_img_map, **val_img_map}, '.')
miss_t = check_files(test_img_map, '.')
print(f"Missing counts -> train+val: {miss_tv}, test: {miss_t}")

print("Setup complete. Next: install torch/timm and implement dataset + baseline model.")

Env check:
$ bash -lc nvidia-smi || true


Sun Sep 28 20:01:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Python: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
CWD: /var/lib/simon/agent_run_states/inaturalist-2019-fgvc6-20250928-191506
Dir listing:
 - .00_eda_and_planning_kernel_state.json (0.00 MB)
 - 00_eda_and_planning.ipynb (0.03 MB)
 - agent_metadata (0.00 MB)
 - artifacts (0.00 MB)
 - description.md (0.01 MB)
 - docker_run.log (0.07 MB)
 - kaggle_sample_submission.csv (0.34 MB)
 - requirements.txt (0.00 MB)
 - submission.csv (0.34 MB)
 - task.txt (0.00 MB)
 - test2019.json (7.86 MB)
 - test2019.tar.gz (9501.32 MB)
 - train2019.json (86.20 MB)
 - train_val (0.00 MB)
 - train_val2019.tar.gz (68612.51 MB)
 - val2019.json (0.84 MB)


Classes: 1010; example cat ids: [0, 1, 2, 3, 4] -> idx [0, 1, 2, 3, 4]
Train images: 232999, Val images: 3030, Test images: 32214
Sample train file_names: ['train_val2019/Plants/400/5a8f865ac7a3b5f7694e3116198c7564.jpg', 'train_val2019/Plants/400/b29ce08f0f5e68cd489ee5e1f1469fcc.jpg', 'train_val2019/Plants/400/545645ddeadacac64926b3bf012916b1.jpg']
Sample val file_names: ['train_val2019/Plants/644/716a69838526f3ada3b2fe2e099cfcb6.jpg', 'train_val2019/Plants/597/0942cc64d2e759c5ee05970d8170942c.jpg', 'train_val2019/Plants/883/acfdbfd9fa675f1c84558e3b9239db90.jpg']
Sample test file_names: ['test2019/2882396373c6e0f89f755fd5e0e810e5.jpg', 'test2019/437b7310fe7d060a8a09a50cd8758d66.jpg', 'test2019/baf96733bdc1eaf9f3ec6eaaea279eb0.jpg']
Train annotations: 232999; unique classes in train: 1010
Val annotations: 3030; unique classes in val: 1010
Top-5 frequent train classes: [(400, 440), (570, 440), (167, 440), (254, 440), (246, 440)]
cat_id 400 -> idx 400
cat_id 570 -> idx 570
cat_id 167 -> i

Insects/24/83162221c209fd4ec0b8a1bf54f9ab4b.jpg
Insects/24/fb04a7b17940580e68d250e30fbd3862.jpg
Insects/24/c0a092b7087f15115776ad6a4ea80a00.jpg
Insects/24/be7505ba1ba1b0081053bde9170938da.jpg
Insects/24/9222de3c41ac78c6224d438749f8f658.jpg



...

Extracting via tar: test2019.tar.gz
$ bash -lc tar -tf test2019.tar.gz | head -n 5


d61dd9d613f7f233584b9a27c503c539.jpg
3161dc4615fe65f1f3ef4aae9b1baacc.jpg
14eea3a6ed04d6c9acfa4a24e508d758.jpg
fa991414d054638b85568142f34968ae.jpg
7e2f14dea7faf00d5dc6cf47bdda0b8a.jpg



$ bash -lc tar -xf test2019.tar.gz


Done in 0.1 min


Checking a few extracted files exist:
Check: train_val2019/Birds/287/b0daafa14d9519bc79d59a40d8628bb2.jpg -> False
Check: train_val2019/Birds/311/5b94cd1749534674a8fc40600b73e880.jpg -> False
Check: train_val2019/Plants/607/e884c86c079c9a80c311f8f21ce08524.jpg -> False
Check: train_val2019/Reptiles/186/b960c9a1b30fe0bb12e92271b29587ea.jpg -> False
Check: train_val2019/Plants/869/2103c419e33bec7199cd01e36690f72b.jpg -> False
Check: test2019/dbd7a4216d5a6db8eb9ed124badf1369.jpg -> False
Check: test2019/6ff2837bf651fa51f6e3710f996d57b9.jpg -> False
Check: test2019/4f7ad6e8b05e1e87caf8072da47cf99f.jpg -> False
Check: test2019/66aa4a19f66392c8917d918f8960bfd2.jpg -> False
Check: test2019/f576b4ce1dc034301093ce6797706ebf.jpg -> False
Missing counts -> train+val: 5, test: 5
Setup complete. Next: install torch/timm and implement dataset + baseline model.


In [3]:
# Fix path prefixes by creating symlinks so JSON file_names resolve
import os
from pathlib import Path

def ensure_prefix_symlink(prefix: str):
    p = Path(prefix)
    if p.exists():
        print(f"Prefix already exists: {prefix}")
        return
    os.symlink('.', prefix)
    print(f"Created symlink: {prefix} -> .")

ensure_prefix_symlink('train_val2019')
ensure_prefix_symlink('test2019')

# Re-check a few paths now
def check_files_again(img_map, root='.'):
    import random
    root = Path(root)
    miss = 0
    keys = list(img_map.keys())
    for k in random.sample(keys, min(5, len(keys))):
        fn = img_map[k]
        fp = root / fn
        ok = fp.exists()
        print(f"Check: {fp} -> {ok}")
        if not ok: miss += 1
    return miss

print("Re-checking after symlinks:")
miss_tv2 = check_files_again({**train_img_map, **val_img_map}, '.')
miss_t2 = check_files_again(test_img_map, '.')
print(f"Missing counts (post-fix) -> train+val: {miss_tv2}, test: {miss_t2}")

Created symlink: train_val2019 -> .
Created symlink: test2019 -> .
Re-checking after symlinks:
Check: train_val2019/Plants/533/e846ebdb3d42e4e7ae34d71f67e18c03.jpg -> True
Check: train_val2019/Birds/261/6bc894a7a2961de674fcadbe4043e28f.jpg -> True
Check: train_val2019/Insects/36/adb367b3b7273dc38c63832a8885af4b.jpg -> True
Check: train_val2019/Plants/835/ebff56ab243284b6ee5e81746c923c42.jpg -> False
Check: train_val2019/Plants/673/b196fda7a666c0a4b1272acc81c42865.jpg -> True
Check: test2019/899d782727cfccfa3936619efe680d67.jpg -> True
Check: test2019/49a4845ff330e901495708b6e5c7a8d2.jpg -> True
Check: test2019/e97add79298145b96bec8eea536ae022.jpg -> True
Check: test2019/058ce0ef598f27e224a950ce56651111.jpg -> True
Check: test2019/707f2a2c659c1e3428b736ad657142ce.jpg -> True
Missing counts (post-fix) -> train+val: 1, test: 0


In [4]:
# Install CUDA 12.1 torch stack and core deps
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('> pip', ' '.join(args), flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any preexisting torch stack to avoid conflicts
for pkg in ('torch','torchvision','torchaudio'):
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)
    except Exception as e:
        print('uninstall error:', e)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Install EXACT cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Freeze versions for later installs
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# Install non-torch deps while honoring constraints
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9',
    'albumentations==1.4.14',
    'opencv-python-headless==4.10.0.84',
    'scikit-learn==1.5.2',
    'numpy', 'pandas',
    '--upgrade-strategy', 'only-if-needed')

# Sanity check GPU
import torch
print('torch:', torch.__version__, 'CUDA build:', getattr(torch.version, 'cuda', None), flush=True)
print('CUDA available:', torch.cuda.is_available(), flush=True)
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0), flush=True)
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'

> pip install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 567.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 468.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 539.9 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 524.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 491.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 498.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 459.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 548.7 MB/s eta 0:00:00


torch: 2.4.1+cu121 CUDA build: 12.1


CUDA available: True


GPU: NVIDIA A10-24Q


In [7]:
# Dataset/Dataloader smoke test with a tiny forward pass (torchvision transforms)
import os, json, math, random, time
from pathlib import Path
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm

with open('artifacts/catid2idx.json','r') as f:
    catid2idx = json.load(f)
idx2catid = {int(v): int(k) for k,v in {k:v for k,v in catid2idx.items()}.items()}

def build_records(j):
    anns_by_img = {}
    for ann in j['annotations']:
        anns_by_img[ann['image_id']] = ann['category_id']
    recs = []
    dropped = 0
    for img in j['images']:
        img_id = img['id']
        fn = img['file_name']
        if img_id in anns_by_img:
            cid = anns_by_img[img_id]
            # map cat id -> idx
            y = catid2idx[str(cid)] if isinstance(catid2idx, dict) and str(cid) in catid2idx else catid2idx[cid]
            # ensure file exists; handle prefix/symlink oddities
            fp = Path(fn)
            if not fp.exists():
                alt = Path(fp.name)
                if alt.exists():
                    fn = str(alt)
                else:
                    dropped += 1
                    continue
            recs.append((img_id, fn, y))
    if dropped:
        print(f"Dropped {dropped} missing files from {len(j['images'])} images.")
    return recs

train_recs = build_records(train_json)
val_recs = build_records(val_json)
print(f"Records -> train: {len(train_recs)}, val: {len(val_recs)}")

IM_SIZE = 224  # sanity run size
IM_MEAN = (0.485, 0.456, 0.406)
IM_STD  = (0.229, 0.224, 0.225)

train_tfms = T.Compose([
    T.RandomResizedCrop(IM_SIZE, scale=(0.4, 1.0), ratio=(0.75, 1.33)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(0.2,0.2,0.2,0.1),
    T.ToTensor(),
    T.Normalize(mean=IM_MEAN, std=IM_STD),
])
val_tfms = T.Compose([
    T.Resize(IM_SIZE, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(IM_SIZE),
    T.ToTensor(),
    T.Normalize(mean=IM_MEAN, std=IM_STD),
])

class INatDataset(Dataset):
    def __init__(self, records, transforms=None):
        self.records = records
        self.transforms = transforms
    def __len__(self):
        return len(self.records)
    def __getitem__(self, idx):
        img_id, fn, y = self.records[idx]
        fp = Path(fn)
        img = Image.open(fp).convert('RGB')
        if self.transforms:
            img = self.transforms(img)
        return img, int(y), int(img_id)

def make_loader(records, transforms, batch_size, shuffle, workers=8):
    ds = INatDataset(records, transforms)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=workers, pin_memory=True, drop_last=False, persistent_workers=(workers>0))
    return ds, dl

# Subsample for smoke test
random.seed(42)
subset_train = random.sample(train_recs, min(128, len(train_recs)))
subset_val = random.sample(val_recs, min(128, len(val_recs)))

train_ds, train_dl = make_loader(subset_train, train_tfms, batch_size=16, shuffle=True, workers=8)
val_ds, val_dl = make_loader(subset_val, val_tfms, batch_size=16, shuffle=False, workers=8)
print("Dataloaders ready. Batches (train/val):", len(train_dl), len(val_dl))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('resnet18', pretrained=True, num_classes=len(catid2idx))
model.to(device)
model.eval()
criterion = torch.nn.CrossEntropyLoss()

@torch.inference_mode()
def smoke_pass(dl):
    t0 = time.time()
    for i, (x,y,ids) in enumerate(dl):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
            logits = model(x)
            loss = criterion(logits, y)
        print(f"batch {i}: x={tuple(x.shape)}, logits={tuple(logits.shape)}, loss={loss.item():.4f}")
        break
    print(f"Smoke pass done in {time.time()-t0:.2f}s")

smoke_pass(train_dl)
smoke_pass(val_dl)
print("Smoke test complete. Next: implement full training loop (convnext_base@384) with class-balanced sampler.")

Dropped 3030 missing files from 3030 images.
Records -> train: 232999, val: 0
Dataloaders ready. Batches (train/val): 8 0


batch 0: x=(16, 3, 224, 224), logits=(16, 1010), loss=7.0293
Smoke pass done in 0.27s


Smoke pass done in 0.09s
Smoke test complete. Next: implement full training loop (convnext_base@384) with class-balanced sampler.


In [12]:
# Train convnext_base@384 with stratified holdout (official val imgs unavailable on disk)
import math, time, random, json
from collections import defaultdict, Counter
from pathlib import Path
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torchvision.transforms as T
import timm
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
from timm.scheduler.cosine_lr import CosineLRScheduler
from timm.utils import ModelEmaV2

torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision('medium')
except Exception:
    pass
random.seed(42); np.random.seed(42); torch.manual_seed(42)

# Reuse simple dataset that reads PIL and applies torchvision transforms
class INatDatasetSimple(Dataset):
    def __init__(self, records, transforms=None):
        self.records = records
        self.transforms = transforms
    def __len__(self):
        return len(self.records)
    def __getitem__(self, idx):
        img_id, fn, y = self.records[idx]
        from PIL import Image
        img = Image.open(fn).convert('RGB')
        if self.transforms: img = self.transforms(img)
        return img, int(y), int(img_id)

# Build 5% stratified holdout (min 3 per class)
labels_to_items = defaultdict(list)
for r in train_recs:
    labels_to_items[r[2]].append(r)
val_frac = 0.05
train_split, valid_split = [], []
random.seed(42)
for y, items in labels_to_items.items():
    if not items: continue
    random.shuffle(items)
    k = max(3, int(len(items)*val_frac))
    valid_split.extend(items[:k])
    train_split.extend(items[k:])
print(f"Holdout sizes -> train: {len(train_split)}, valid: {len(valid_split)} (val_frac={val_frac})")

IM_SIZE = 384
IM_MEAN = (0.485, 0.456, 0.406)
IM_STD  = (0.229, 0.224, 0.225)
train_tfms = T.Compose([
    T.RandomResizedCrop(IM_SIZE, scale=(0.4,1.0), ratio=(0.75,1.33)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(num_magnitude_bins=31),
    T.ToTensor(),
    T.RandomErasing(p=0.25, value='random'),
    T.Normalize(IM_MEAN, IM_STD),
])
val_tfms = T.Compose([
    T.Resize(int(IM_SIZE/0.875), interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(IM_SIZE),
    T.ToTensor(),
    T.Normalize(IM_MEAN, IM_STD),
])

# Datasets
train_ds = INatDatasetSimple(train_split, train_tfms)
valid_ds = INatDatasetSimple(valid_split, val_tfms)

# Class-balanced sampler ~ 1/sqrt(freq)
cls_counts = Counter([y for _,_,y in train_split])
weights = [1.0/np.sqrt(cls_counts[y]) for _,_,y in train_split]
sampler = WeightedRandomSampler(weights, num_samples=len(train_split), replacement=True)

BATCH_SIZE = 48  # adjust if OOM; fallback 32
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, shuffle=False, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=6)
valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=6)
print("DL ready:", len(train_dl), len(valid_dl))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(catid2idx)
# Prefer IN-21k finetuned head
model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=True, num_classes=num_classes)
model.to(device)
model = model.to(memory_format=torch.channels_last)

# Mixup/CutMix + SoftTarget loss
mixup_fn = Mixup(mixup_alpha=0.3, cutmix_alpha=0.5, prob=1.0, switch_prob=0.5, label_smoothing=0.0, num_classes=num_classes)
criterion_train = SoftTargetCrossEntropy()
criterion_eval = nn.CrossEntropyLoss()

fused_ok = (device.type=='cuda')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.05, betas=(0.9,0.999), fused=fused_ok)
EPOCHS = 15  # trimmed to fit within time budget
steps_per_epoch = max(1, len(train_dl))
total_updates = EPOCHS * steps_per_epoch
warmup_t = 1000  # updates-based warmup
sched = CosineLRScheduler(
    optimizer,
    t_initial=total_updates,
    lr_min=1e-6,
    warmup_t=warmup_t,
    warmup_lr_init=1e-6,
    k_decay=1.0,
    t_in_epochs=False
)

scaler = torch.amp.GradScaler('cuda', enabled=(device.type=='cuda'))
ema = ModelEmaV2(model, decay=0.9998, device=device if device.type=='cuda' else None)

def evaluate(use_ema=True):
    m = ema.module if use_ema and ema is not None else model
    m.eval()
    correct = 0; total = 0; loss_sum = 0.0
    t0 = time.time()
    with torch.no_grad():
        for bx, (x,y,ids) in enumerate(valid_dl):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                logits = m(x)
                loss = criterion_eval(logits, y)
            loss_sum += loss.item() * y.size(0)
            pred = logits.argmax(dim=1)
            correct += (pred==y).sum().item()
            total += y.size(0)
            if (bx+1)%50==0:
                print(f"  Eval batch {bx+1}/{len(valid_dl)}", flush=True)
    acc = correct/max(1,total)
    print(f"Eval done in {time.time()-t0:.1f}s; acc={acc:.4f}; loss={loss_sum/max(1,total):.4f}")
    return acc, loss_sum/max(1,total)

best_acc = -1.0
t_start = time.time()
num_updates = 0
for ep in range(EPOCHS):
    # Taper mixup late
    if ep >= EPOCHS - 2:
        mixup_fn.prob = 0.5
    model.train()
    t0 = time.time()
    running = 0.0; n_seen = 0
    for bi, (x,y,ids) in enumerate(train_dl):
        x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        y = y.to(device, non_blocking=True)
        # apply mixup
        x, y_soft = mixup_fn(x, y)
        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
            logits = model(x)
            loss = criterion_train(logits, y_soft)
        scaler.scale(loss).backward()
        # unscale and clip before stepping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        num_updates += 1
        sched.step_update(num_updates)
        if num_updates > warmup_t:
            ema.update(model)
        running += loss.item()*x.size(0); n_seen += x.size(0)
        if (bi+1)%100==0:
            cur_lr = optimizer.param_groups[0]['lr']
            print(f"ep {ep+1}/{EPOCHS} batch {bi+1}/{len(train_dl)} lr={cur_lr:.2e} loss={running/max(1,n_seen):.4f} elapsed={time.time()-t0:.1f}s", flush=True)
    print(f"Epoch {ep+1} train_loss={running/max(1,n_seen):.4f} epoch_time={time.time()-t0:.1f}s total_elapsed={(time.time()-t_start)/60:.1f}m")
    acc, vloss = evaluate(use_ema=True)
    if acc>best_acc:
        best_acc = acc
        torch.save({'model': ema.module.state_dict(), 'acc': acc}, 'best_convnext_baseline.pt')
        print(f"Saved new best (EMA) acc={acc:.4f}")

print("Training finished. Best holdout acc:", best_acc)
print("Next: add TTA inference on test and write submission.")

Holdout sizes -> train: 221594, valid: 11405 (val_frac=0.05)


DL ready: 4617 238


ep 1/15 batch 100/4617 lr=1.01e-04 loss=6.9130 elapsed=56.4s


ep 1/15 batch 200/4617 lr=2.01e-04 loss=6.4803 elapsed=112.8s


ep 1/15 batch 300/4617 lr=3.01e-04 loss=5.9256 elapsed=169.8s


...

  Eval batch 50/238


  Eval batch 100/238


  Eval batch 150/238


  Eval batch 200/238


Eval done in 46.5s; acc=0.8401; loss=0.5888
Training finished. Best holdout acc: 0.8404208680403332
Next: add TTA inference on test and write submission.


In [ ]:
# Inference & Submission: TTA on test, write submission.csv
import json, time, math, torch, gc
from pathlib import Path
from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
import timm

torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

# Clean GPU memory before inference
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load mappings
with open('artifacts/idx2catid.json','r') as f:
    idx2catid = {int(k): int(v) for k,v in json.load(f).items()}

# Build test records (filter missing)
def build_test_records(j):
    recs = []
    miss = 0
    for img in j['images']:
        img_id = img['id']
        fn = img['file_name']
        fp = Path(fn)
        if not fp.exists():
            alt = Path(fp.name)
            if alt.exists():
                fn = str(alt)
            else:
                miss += 1
                continue
        recs.append((img_id, fn))
    if miss:
        print(f"[WARN] Dropped {miss} missing test files")
    return recs

test_recs = build_test_records(test_json)
print(f"Test records: {len(test_recs)}")

class TestDS(Dataset):
    def __init__(self, records, tfm):
        self.records = records
        self.tfm = tfm
    def __len__(self): return len(self.records)
    def __getitem__(self, i):
        img_id, fn = self.records[i]
        img = Image.open(fn).convert('RGB')
        x = self.tfm(img)
        return x, int(img_id)

def make_eval_tfms(sz):
    return T.Compose([
        T.Resize(int(sz/0.875), interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(sz),
        T.ToTensor(),
        T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ])

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = len(idx2catid)

# Select best available checkpoint (prioritize averaged checkpoints)
ckpt_candidates = [
    Path('avg_ema_last_448.pt'),
    Path('avg_ema_last.pt'),
    Path('best_convnext_baseline_cont.pt'),
    Path('best_convnext_baseline.pt'),
]
ckpt_path = next((p for p in ckpt_candidates if p.exists()), None)
assert ckpt_path is not None, 'No checkpoint found. Train first.'
print('Using checkpoint:', ckpt_path)

# Use the exact same architecture variant as training
model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=False, num_classes=NUM_CLASSES)
sd = torch.load(ckpt_path, map_location='cpu')
state = sd['model'] if isinstance(sd, dict) and 'model' in sd else sd
missing, unexpected = model.load_state_dict(state, strict=False)
if missing or unexpected:
    print('[WARN] load_state_dict mismatch -> missing:', len(missing), 'unexpected:', len(unexpected))
model.to(DEVICE).eval()
model = model.to(memory_format=torch.channels_last)

# TTA settings per plan: multi-scale with hflip
scales = [384, 416]
use_hflip = True
bs = 32  # OOM-safe on A10 for 384/416

all_ids = [rid for rid,_ in test_recs]
all_logits = torch.zeros((len(test_recs), NUM_CLASSES), dtype=torch.float32, device='cpu')

@torch.no_grad()
def infer_one_scale(sz, flip=False):
    tfm = make_eval_tfms(sz)
    ds = TestDS(test_recs, tfm)
    dl = DataLoader(ds, batch_size=bs, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    idx = 0
    t0 = time.time()
    for b, (x, ids) in enumerate(dl):
        x = x.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
        if flip:
            x = TF.hflip(x)
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=DEVICE.type=='cuda'):
            logits = model(x)
        n = logits.shape[0]
        all_logits[idx:idx+n] += logits.detach().to('cpu')
        idx += n
        if (b+1)%100==0:
            print(f"  INF sz={sz} flip={flip} batch {b+1}/{len(dl)}", flush=True)
    print(f"  Done sz={sz} flip={flip} in {time.time()-t0:.1f}s")

t_start = time.time()
print('Inference scales:', scales, 'hflip:', use_hflip, 'bs:', bs)
for sz in scales:
    infer_one_scale(sz, flip=False)
    if use_hflip:
        infer_one_scale(sz, flip=True)
num_passes = len(scales) * (2 if use_hflip else 1)
print(f"Inference complete in {(time.time()-t_start)/60:.1f}m; passes={num_passes}")

# Average logits by number of passes
all_logits /= max(1, num_passes)
pred_idx = all_logits.argmax(dim=1).tolist()

# Map to original category_id and write submission
id_to_pred = {}
for i, (img_id, _) in enumerate(test_recs):
    id_to_pred[img_id] = idx2catid[int(pred_idx[i])]

# Ensure all test ids present; if any missing due to file issues, fill with most frequent train class id (idx 0 as fallback)
all_test_ids = [img['id'] for img in test_json['images']]
fallback_cid = idx2catid[0]
rows = []
for img_id in all_test_ids:
    cid = id_to_pred.get(img_id, fallback_cid)
    rows.append((img_id, cid))
rows.sort(key=lambda x: x[0])

with open('submission.csv','w') as f:
    f.write('id,predicted\n')
    for img_id, cid in rows:
        f.write(f"{img_id},{cid}\n")
print('Wrote submission.csv with', len(rows), 'rows')
print('Head:')
print('\n'.join(open('submission.csv').read().splitlines()[:5]))

Test records: 32214
Using checkpoint: avg_ema_last.pt


Inference scales: [384, 416] hflip: True bs: 32


/tmp/ipykernel_222/4093238535.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(ckpt_path, map_location='cpu')


In [15]:
# Continuation: extend training to 25 epochs and prep averaged EMA weights
import os, time, json, math, torch, glob, gc
from pathlib import Path
from torch import nn
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import timm
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
from timm.scheduler.cosine_lr import CosineLRScheduler
from timm.utils import ModelEmaV2

# Enable TF32 for throughput on A10
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(catid2idx)

def build_loaders_for_continuation(im_size=384, batch_size=48, workers=12, erase_p=0.25):
    train_tf = T.Compose([
        T.RandomResizedCrop(im_size, scale=(0.4,1.0), ratio=(0.75,1.33)),
        T.RandomHorizontalFlip(p=0.5),
        T.TrivialAugmentWide(num_magnitude_bins=31),
        T.ToTensor(),
        T.RandomErasing(p=erase_p, value='random'),
        T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ])
    val_tf = T.Compose([
        T.Resize(int(im_size/0.875), interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(im_size),
        T.ToTensor(),
        T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ])
    train_ds = INatDatasetSimple(train_split, train_tf)
    val_ds = INatDatasetSimple(valid_split, val_tf)
    from collections import Counter
    cls_counts = Counter([y for _,_,y in train_split])
    weights = [1.0/np.sqrt(cls_counts[y]) for _,_,y in train_split]
    sampler = WeightedRandomSampler(weights, num_samples=len(train_split), replacement=True)
    train_dl = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, shuffle=False, num_workers=workers, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    return train_dl, val_dl

def evaluate_model(m, dl):
    m.eval(); correct=0; total=0; loss_sum=0.0
    ce = nn.CrossEntropyLoss()
    t0=time.time()
    with torch.no_grad():
        for bx,(x,y,ids) in enumerate(dl):
            x=x.to(device, non_blocking=True).to(memory_format=torch.channels_last); y=y.to(device, non_blocking=True)
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                logits=m(x); loss=ce(logits,y)
            loss_sum+=loss.item()*y.size(0); pred=logits.argmax(1); correct+=(pred==y).sum().item(); total+=y.size(0)
    acc=correct/max(1,total); vloss=loss_sum/max(1,total)
    print(f"[CONT] Eval acc={acc:.4f} loss={vloss:.4f} time={time.time()-t0:.1f}s")
    return acc, vloss

def average_state_dicts(paths):
    assert paths, 'No checkpoint paths for averaging'
    avg=None; n=0
    for p in paths:
        sd=torch.load(p, map_location='cpu')
        state=sd['model'] if isinstance(sd, dict) and 'model' in sd else sd
        if avg is None:
            avg={k: v.clone().float() for k,v in state.items()}
        else:
            for k in avg.keys():
                avg[k] += state[k].float()
        n+=1
    for k in avg.keys():
        avg[k] /= n
    return avg

def free_cuda_memory():
    for name in ('model','ema','optimizer','scaler','train_dl','valid_dl','train_ds','valid_ds'):
        if name in globals():
            try: del globals()[name]
            except: pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def continue_training(extra_epochs=10, peak_lr=1e-3, ema_decay=0.9999, warmup_updates=300, save_last_n=8, batch_size=40, workers=12):
    free_cuda_memory()
    print(f"[CONT] Loading best checkpoint and continuing for {extra_epochs} epochs")
    ckpt = torch.load('best_convnext_baseline.pt', map_location='cpu')
    model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=False, num_classes=num_classes)
    model.load_state_dict(ckpt['model'], strict=True)
    model.to(device).train()
    model = model.to(memory_format=torch.channels_last)
    ema = ModelEmaV2(model, decay=ema_decay, device=device if device.type=='cuda' else None)
    # Initialize EMA with current weights
    for ema_v, model_v in zip(ema.module.state_dict().values(), model.state_dict().values()):
        ema_v.copy_(model_v)

    # Build loaders (RandomErasing on initially; will disable in final epoch below)
    train_dl, val_dl = build_loaders_for_continuation(im_size=384, batch_size=batch_size, workers=workers, erase_p=0.25)

    optimizer = torch.optim.AdamW(model.parameters(), lr=peak_lr, weight_decay=0.05, betas=(0.9,0.999), fused=(device.type=='cuda'))
    steps_per_epoch = len(train_dl)
    total_updates = extra_epochs * steps_per_epoch
    sched = CosineLRScheduler(optimizer, t_initial=total_updates, lr_min=1e-6, warmup_t=warmup_updates, warmup_lr_init=1e-6, k_decay=1.0, t_in_epochs=False)
    scaler = torch.amp.GradScaler('cuda', enabled=(device.type=='cuda'))

    mixup_fn = Mixup(mixup_alpha=0.3, cutmix_alpha=0.5, prob=1.0, switch_prob=0.5, label_smoothing=0.0, num_classes=num_classes)
    criterion_soft = SoftTargetCrossEntropy()
    criterion_hard = nn.CrossEntropyLoss(label_smoothing=0.1)

    best_acc = -1.0
    num_updates = 0
    last_ckpts = []
    t_start = time.time()
    for ep in range(extra_epochs):
        is_last = (ep == extra_epochs-1)
        # Taper mixup: last 3 epochs prob=0.5; final epoch off
        if is_last:
            mixup_fn.prob = 0.0
        elif ep >= extra_epochs - 3:
            mixup_fn.prob = 0.5
        else:
            mixup_fn.prob = 1.0
        # Disable RandomErasing in final epoch by rebuilding train loader with erase_p=0
        if is_last:
            train_dl, _ = build_loaders_for_continuation(im_size=384, batch_size=batch_size, workers=workers, erase_p=0.0)
        model.train(); t0 = time.time(); running=0.0; n_seen=0
        for bi, (x,y,ids) in enumerate(train_dl):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            if mixup_fn.prob > 0:
                x, y_soft = mixup_fn(x, y)
                with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                    logits = model(x)
                    loss = criterion_soft(logits, y_soft)
            else:
                with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                    logits = model(x)
                    loss = criterion_hard(logits, y)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer); scaler.update()
            num_updates += 1; sched.step_update(num_updates)
            if num_updates > warmup_updates:
                ema.update(model)
            running += loss.item()*x.size(0); n_seen += x.size(0)
            if (bi+1)%100==0:
                cur_lr = optimizer.param_groups[0]['lr']
                print(f"[CONT] ep {ep+1}/{extra_epochs} batch {bi+1}/{len(train_dl)} lr={cur_lr:.2e} loss={running/max(1,n_seen):.4f} elapsed={time.time()-t0:.1f}s", flush=True)
        print(f"[CONT] Epoch {ep+1} train_loss={running/max(1,n_seen):.4f} time={time.time()-t0:.1f}s total={(time.time()-t_start)/60:.1f}m")
        acc, vloss = evaluate_model(ema.module, val_dl)
        # Save rolling last-N EMA checkpoints
        outp = f"ema_cont_ep{ep+1}.pt"
        torch.save({'model': ema.module.state_dict(), 'acc': acc}, outp)
        last_ckpts.append(outp)
        if len(last_ckpts) > save_last_n:
            rm = last_ckpts.pop(0)
            try: os.remove(rm)
            except: pass
        if acc > best_acc:
            best_acc = acc
            torch.save({'model': ema.module.state_dict(), 'acc': acc}, 'best_convnext_baseline_cont.pt')
            print(f"[CONT] New best acc={acc:.4f}")
    print('[CONT] Done. Best acc:', best_acc)
    print('[CONT] Averaging last', len(last_ckpts), 'EMA checkpoints...')
    avg_sd = average_state_dicts(last_ckpts) if last_ckpts else ema.module.state_dict()
    torch.save({'model': avg_sd, 'acc': best_acc}, 'avg_ema_last.pt')
    print('[CONT] Saved avg_ema_last.pt')

print('Continuation cell ready. After the initial 15-epoch run finishes, execute this cell to extend to 25 epochs and produce avg_ema_last.pt for TTA.')

Continuation cell ready. After the initial 15-epoch run finishes, execute this cell to extend to 25 epochs and produce avg_ema_last.pt for TTA.


In [17]:
# Execute continuation (extend training) after initial 15-epoch run completes
# This will load best_convnext_baseline.pt and run extra epochs with EMA=0.9999,
# taper mixup, final-epoch CE+LS and RandomErasing off, and save/average last-N EMA checkpoints.
try:
    import os
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    # Adjust extra_epochs if needed to hit ~22-25 total
    # Reduce batch_size and workers to avoid CUDA OOM
    continue_training(extra_epochs=10, peak_lr=1e-3, ema_decay=0.9999, warmup_updates=300, save_last_n=8, batch_size=32, workers=8)
except NameError:
    print("Continuation functions not defined yet. Run cell 7 first.")

[CONT] Loading best checkpoint and continuing for 10 epochs


/tmp/ipykernel_222/2104107370.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('best_convnext_baseline.pt', map_location='cpu')


[CONT] ep 1/10 batch 100/6925 lr=3.34e-04 loss=1.3814 elapsed=49.4s


[CONT] ep 1/10 batch 200/6925 lr=6.67e-04 loss=1.5789 elapsed=87.9s


[CONT] ep 1/10 batch 300/6925 lr=1.00e-03 loss=1.9061 elapsed=126.9s


...

[CONT] ep 8/10 batch 6800/6925 lr=9.81e-05 loss=1.8367 elapsed=2805.8s


[CONT] ep 8/10 batch 6900/6925 lr=9.67e-05 loss=1.8357 elapsed=2847.1s


[CONT] Epoch 8 train_loss=1.8341 time=2857.4s total=386.2m


[CONT] Eval acc=0.8150 loss=0.6502 time=47.1s


KeyboardInterrupt: 

In [ ]:
# 448 fine-tune: load averaged EMA, 2 epochs, eval-style augs, small LR, save avg EMA
import time, torch
from pathlib import Path
from torch import nn
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import timm
from timm.scheduler.cosine_lr import CosineLRScheduler
from timm.utils import ModelEmaV2

def build_loaders_448(batch_size=24, workers=8):
    sz = 448
    train_tf = T.Compose([
        T.Resize(int(sz/0.875), interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(sz),
        T.RandomHorizontalFlip(p=0.5),
        T.ToTensor(),
        T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ])
    val_tf = T.Compose([
        T.Resize(int(sz/0.875), interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(sz),
        T.ToTensor(),
        T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ])
    train_ds = INatDatasetSimple(train_split, train_tf)
    val_ds = INatDatasetSimple(valid_split, val_tf)
    from collections import Counter
    cls_counts = Counter([y for _,_,y in train_split])
    weights = [1.0/np.sqrt(cls_counts[y]) for _,_,y in train_split]
    sampler = WeightedRandomSampler(weights, num_samples=len(train_split), replacement=True)
    train_dl = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, shuffle=False, num_workers=workers, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    return train_dl, val_dl

def evaluate_448(m, dl):
    m.eval(); correct=0; total=0; loss_sum=0.0
    ce = nn.CrossEntropyLoss()
    t0=time.time()
    with torch.no_grad():
        for bx,(x,y,ids) in enumerate(dl):
            x=x.to(device, non_blocking=True).to(memory_format=torch.channels_last); y=y.to(device, non_blocking=True)
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                logits=m(x); loss=ce(logits,y)
            loss_sum+=loss.item()*y.size(0); pred=logits.argmax(1); correct+=(pred==y).sum().item(); total+=y.size(0)
    acc=correct/max(1,total); vloss=loss_sum/max(1,total)
    print(f"[FT448] Eval acc={acc:.4f} loss={vloss:.4f}")
    return acc, vloss

def average_sd(paths):
    avg=None; n=0
    for p in paths:
        sd=torch.load(p, map_location='cpu')
        state=sd['model'] if isinstance(sd, dict) and 'model' in sd else sd
        if avg is None:
            avg={k: v.clone().float() for k,v in state.items()}
        else:
            for k in avg.keys():
                avg[k] += state[k].float()
        n+=1
    for k in avg.keys(): avg[k]/=max(1,n)
    return avg

def fine_tune_448(epochs=1, lr=2e-5, wd=0.05, ema_decay=0.9999, warmup_updates=50):
    print('[FT448] Loading avg_ema_last.pt for 448 fine-tune...')
    ckpt_path = Path('avg_ema_last.pt')
    assert ckpt_path.exists(), 'avg_ema_last.pt not found; run continuation first.'
    sd = torch.load(ckpt_path, map_location='cpu')
    model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=False, num_classes=len(catid2idx))
    missing, unexpected = model.load_state_dict(sd['model'], strict=False)
    if missing or unexpected:
        print('[FT448][WARN] load_state_dict mismatch -> missing:', len(missing), 'unexpected:', len(unexpected))
    model.to(device).train()
    model = model.to(memory_format=torch.channels_last)
    ema = ModelEmaV2(model, decay=ema_decay, device=device if device.type=='cuda' else None)
    for ev, mv in zip(ema.module.state_dict().values(), model.state_dict().values()): ev.copy_(mv)

    train_dl, val_dl = build_loaders_448(batch_size=24, workers=8)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=(0.9,0.999), fused=(device.type=='cuda'))
    steps_per_epoch = len(train_dl)
    total_updates = epochs * steps_per_epoch
    sched = CosineLRScheduler(optimizer, t_initial=total_updates, lr_min=1e-6, warmup_t=warmup_updates, warmup_lr_init=1e-6, k_decay=1.0, t_in_epochs=False)
    scaler = torch.amp.GradScaler('cuda', enabled=(device.type=='cuda'))
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    best = -1.0; num_updates=0; saved=[]
    for ep in range(epochs):
        t0=time.time(); run=0.0; n=0
        for bi,(x,y,ids) in enumerate(train_dl):
            x=x.to(device, non_blocking=True).to(memory_format=torch.channels_last); y=y.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=device.type=='cuda'):
                logits=model(x); loss=criterion(logits,y)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer); scaler.update()
            num_updates+=1; sched.step_update(num_updates); ema.update(model)
            run+=loss.item()*x.size(0); n+=x.size(0)
            if (bi+1)%100==0:
                print(f"[FT448] ep {ep+1}/{epochs} batch {bi+1}/{len(train_dl)} lr={optimizer.param_groups[0]['lr']:.2e} loss={run/max(1,n):.4f}", flush=True)
        print(f"[FT448] Epoch {ep+1} train_loss={run/max(1,n):.4f} time={time.time()-t0:.1f}s")
        acc,_ = evaluate_448(ema.module, val_dl)
        outp=f"ema_ft448_ep{ep+1}.pt"
        torch.save({'model': ema.module.state_dict(), 'acc': acc}, outp); saved.append(outp)
        if acc>best: best=acc; torch.save({'model': ema.module.state_dict(), 'acc': acc}, 'best_ft448.pt')
        # Optional: add a 2nd epoch only if improved
        if ep==0 and epochs>=2 and best<0.0001:
            print('[FT448] No improvement after 1 epoch; stopping early.')
            break
    print('[FT448] Averaging', len(saved), 'EMA checkpoints...')
    avg = average_sd(saved)
    torch.save({'model': avg, 'acc': best}, 'avg_ema_last_448.pt')
    print('[FT448] Saved avg_ema_last_448.pt (prioritize this in inference).')

print('FT-448 cell ready. After continuation produces avg_ema_last.pt, run fine_tune_448() then inference (update TTA to include 448).')

In [18]:
# Utility: Average saved continuation EMA checkpoints into avg_ema_last.pt
import glob, re, torch
from pathlib import Path

def average_saved_ema(last_n=8, pattern='ema_cont_ep*.pt', out_path='avg_ema_last.pt'):
    paths = sorted(glob.glob(pattern), key=lambda p: int(re.search(r'ep(\d+)', p).group(1)) if re.search(r'ep(\d+)', p) else -1)
    if not paths:
        print('[AVG] No ema_cont_ep*.pt files found.')
        return
    if last_n is not None and last_n > 0:
        paths = paths[-last_n:]
    print('[AVG] Averaging the following checkpoints:', paths)
    avg=None; n=0
    for p in paths:
        sd=torch.load(p, map_location='cpu')
        state=sd['model'] if isinstance(sd, dict) and 'model' in sd else sd
        if avg is None:
            avg={k: v.clone().float() for k,v in state.items()}
        else:
            for k in avg.keys():
                avg[k] += state[k].float()
        n+=1
    for k in avg.keys():
        avg[k] /= max(1,n)
    torch.save({'model': avg, 'acc': None, 'averaged_n': n}, out_path)
    print(f"[AVG] Saved {out_path} from {n} checkpoints")

print('Averaging utility ready. If you stop continuation early (e.g., after 8-9 epochs), run average_saved_ema(last_n=6 or 8).')

Averaging utility ready. If you stop continuation early (e.g., after 8-9 epochs), run average_saved_ema(last_n=6 or 8).


In [20]:
# Average last 8 EMA checkpoints now (fix glob shadowing)
import importlib
import glob as glob_module
globals()['glob'] = glob_module  # ensure average_saved_ema sees the module, not a function
import os
print("Found EMA ckpts:", sorted(glob_module.glob('ema_cont_ep*.pt')))
try:
    average_saved_ema(last_n=8, pattern='ema_cont_ep*.pt', out_path='avg_ema_last.pt')
except Exception as e:
    import traceback; traceback.print_exc()
print("avg_ema_last.pt exists:", os.path.exists('avg_ema_last.pt'))

Found EMA ckpts: ['ema_cont_ep1.pt', 'ema_cont_ep2.pt', 'ema_cont_ep3.pt', 'ema_cont_ep4.pt', 'ema_cont_ep5.pt', 'ema_cont_ep6.pt', 'ema_cont_ep7.pt', 'ema_cont_ep8.pt']
[AVG] Averaging the following checkpoints: ['ema_cont_ep1.pt', 'ema_cont_ep2.pt', 'ema_cont_ep3.pt', 'ema_cont_ep4.pt', 'ema_cont_ep5.pt', 'ema_cont_ep6.pt', 'ema_cont_ep7.pt', 'ema_cont_ep8.pt']


/tmp/ipykernel_222/3711789468.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd=torch.load(p, map_location='cpu')


[AVG] Saved avg_ema_last.pt from 8 checkpoints
avg_ema_last.pt exists: True
